In [1]:
import numpy as np
import cv2

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [2]:
from tensorflow.keras.datasets import mnist

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0


X_train_flat = X_train.reshape(-1, 28*28)
X_test_flat = X_test.reshape(-1, 28*28)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
svm_raw = LinearSVC(max_iter=10000)
svm_raw.fit(X_train_flat, y_train)

y_pred_raw = svm_raw.predict(X_test_flat)
acc_raw = accuracy_score(y_test, y_pred_raw)

In [5]:
confusion_matrix(y_test, y_pred_raw)

array([[ 962,    0,    2,    1,    1,    4,    5,    3,    1,    1],
       [   0, 1112,    3,    2,    0,    1,    5,    1,   11,    0],
       [  11,   11,  914,   18,   10,    4,   13,   12,   36,    3],
       [   4,    0,   19,  918,    2,   22,    5,   12,   20,    8],
       [   1,    4,    5,    4,  913,    0,    9,    3,    5,   38],
       [   9,    2,    0,   39,   12,  768,   18,    7,   29,    8],
       [   7,    4,    7,    2,    5,   21,  909,    1,    2,    0],
       [   2,    8,   23,    5,    7,    1,    1,  948,    4,   29],
       [  11,   13,    8,   21,   14,   31,    8,   13,  843,   12],
       [   7,    8,    2,   15,   31,   12,    0,   26,   12,  896]])

In [10]:
def openCVHOG(im):
    im = cv2.resize(im, (20, 20))
    im_uint8 = (im * 255).astype('uint8')
    winSize = (20, 20)
    blockSize = (10, 10)
    blockStride = (5, 5)
    cellSize = (10, 10)
    nbins = 9
    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
    descriptor = np.ravel(hog.compute(im_uint8))
    return descriptor

In [11]:
n_train = 10000
n_test = 2000

In [12]:
X_train_sample = X_train[:n_train]
y_train_sample = y_train[:n_train]
X_test_sample = X_test[:n_test]
y_test_sample = y_test[:n_test]

In [13]:
X_train_hog = np.array([openCVHOG(im) for im in X_train_sample])
X_test_hog = np.array([openCVHOG(im) for im in X_test_sample])

In [14]:
X_train_hog.shape[1]

81

Czyli zklasyfikowano 81 cech

In [15]:
svm_hog = LinearSVC(max_iter=10000)
svm_hog.fit(X_train_hog, y_train_sample)

LinearSVC(max_iter=10000)

In [16]:
y_pred_hog = svm_hog.predict(X_test_hog)
acc_hog = accuracy_score(y_test_sample, y_pred_hog)

Dokładność (HOG) features

In [17]:
acc_hog

0.9505

In [18]:
confusion_matrix(y_test_sample, y_pred_hog)

array([[174,   0,   0,   0,   0,   0,   0,   0,   1,   0],
       [  1, 229,   1,   0,   2,   0,   0,   1,   0,   0],
       [  1,   0, 214,   2,   1,   0,   1,   0,   0,   0],
       [  0,   0,   1, 199,   2,   3,   0,   2,   0,   0],
       [  0,   0,   3,   1, 200,   0,   5,   1,   1,   6],
       [  0,   0,   1,   4,   0, 173,   0,   0,   1,   0],
       [  2,   1,   0,   0,   0,   2, 172,   0,   0,   1],
       [  0,   1,   8,   2,   3,   0,   0, 185,   3,   3],
       [  2,   1,   2,   4,   3,   0,   0,   1, 177,   2],
       [  0,   1,   0,   5,   5,   0,   0,   0,   5, 178]])

In [19]:
#Porównanie modeli
acc_raw
acc_hog

0.9505